In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pystan
from pystan import StanModel 
from numpy import polyval, place, extract, any, asarray, nan, inf, pi
from numpy import (where, arange, putmask, ravel, sum, shape,
                   log, sqrt, exp, arctanh, tan, sin, arcsin, arctan,
                   tanh, cos, cosh, sinh, log1p, expm1)

from scipy.stats import rv_continuous
from scipy.stats import f


class frechet_gen(rv_continuous):
#     def _argcheck(self, c):
#         c = asarray(c)
#         self.b = where(c < 0, 1.0/abs(c), inf)
#         return where(c == 0, 0, 1)

#     def _pdf(self, x, alpha1, alpha2, beta):
#         Px = 1 / beta / ss.beta(alpha1, alpha2) * pow(x/beta, asarray(alpha1-1.0)) * pow(1 + x/beta, asarray(- alpha1 - alpha2))
#         return Px

#     def _logpdf(self, x, alpha1, alpha2, beta):
#         return (alpha1 - 1) * np.log(x) - alpha1 * np.log(beta) - np.log(ss.beta(alpha1, alpha2)) - (alpha1 + alpha2) * np.log(1 + x/beta)

    def _cdf(self, x, beta):
        return exp(-pow(x, -1/beta))
#     def _ppf(self, q, c):
#         vals = 1.0/c * (pow(1-q, -c)-1)
#         return vals

#     def _munp(self, n, c):
#         k = arange(0, n+1)
#         val = (-1.0/c)**n * sum(comb(n, k)*(-1)**k / (1.0-c*k), axis=0)
#         return where(c*n < 1, val, inf)

#     def _entropy(self, c):
#         if (c > 0):
#             return 1+c
#         else:
#             self.b = -1.0 / c
#             return rv_continuous._entropy(self, c)
frechet = frechet_gen(a=0.0, name='frechet') # we specify the support [a,b], no b means b = infinity

In [ ]:
# here we are able to sample rv exactly as in publication
N, beta = 1000,  1/2 # run with n = 1000000
# beta = 1/2 because we want to have a finite mean and for this beta we could check that the code is good
r = frechet.rvs(beta, size=N)

In [ ]:
# histogram of data
fig, ax = plt.subplots(figsize=[8,6])

# set plot title
ax.set_title("Data sampled from Frechet ")

# set x-axis name
ax.set_xlabel("value")

# set y-axis name
ax.set_ylabel("number of records")

# create histogram within output
Nb, bins, patches = ax.hist(r, bins=100, color="#777777") #initial color of all bins

# for bin_size, bin, patch in zip(Nb, bins, patches):
#     if bin_size == 100:
#         patch.set_facecolor("#FF000")
#         patch.set_label("something")

plt.show()

In [ ]:
from prettytable import PrettyTable
m = 3 # how much we round
t = PrettyTable(['pr.distr.', 'mean', 'sd', '92.5%', '95%', '97.5%', '99%', '99.9%'])
t.add_row(['Fréchet',
           "%.3f" % round(np.mean(r),m),
           "%.3f" % round(np.std(r),m),
#            "%.3f" % round(np.percentile(r, 2.5),m), 
#            "%.3f" % round(np.percentile(r, 25),m), 
#            "%.3f" % round(np.percentile(r, 50),m), 
           "%.3f" % round(np.percentile(r, 92.5),m), 
           "%.3f" % round(np.percentile(r, 95),m), 
           "%.3f" % round(np.percentile(r, 97.5),m), 
           "%.3f" % round(np.percentile(r, 99),m),
           "%.3f" % round(np.percentile(r, 99.9),m)])
# t.add_row(['Bob', 19])
print(t)

In [2]:
# we need a function to get a excesses
def k_greatest_values(a,k):
    """returns k greatest elements from the list and k-1 value starting from which we consider values to be extreme"""
    u = np.sort(a, axis=None)[-1-k]
    a = np.sort(a, axis=None)[-1-k+1:]
    a = asarray([a-u for x in a])
    return(a[1].tolist(), u) # u the starting value from which we consider others as excesses

In [ ]:
#  function to make plots of traceplot and horizontal histogram of the traceplot in one row
def plot(traceplot, param, distr):
    fig, ax = plt.subplots(1, 2, sharey=True, figsize=[14,8])
    # set x-axis name
    ax[0].set_xlabel("number of iteration")
    ax[0].plot(traceplot)
    ax[0].set_ylabel("value of " + param)
    # set y-axis name
    ax[1].set_xlabel("quantity of records")
    # ax[1].set_ylabel("number of records")

    plt.suptitle('traceplot with histogram of values of alpha parameter in ' + distr + ' fitted to the excesses from Fréchet')
    # titles of subplots, here we don't use it 
    # ax[0].set_title("traceplot of beta in GPD(alpha, beta)")
    # ax[1].set_title("Values of beta in GPD(alpha, beta) fitted to the excesses from Frechet ")

    # create histogram within output
    Nb, bins, patches = ax[1].hist(traceplot, bins=50, color="#777777",  orientation="horizontal") #initial color of all bins
    return(plt.show())

In [ ]:
k = 100 # number of excesses
frechet, u = k_greatest_values(r, k)
# frechet # so we recover  k = 100 excesses sampled from Frechet distribution

In [3]:
GPD = """
functions {
  real myGPD_lpdf(real y, real alpha, real beta) {
      return -(alpha + 1)*( log(1+y/beta) )+(log(alpha) - log(beta));
  }
  
  real myBetaPrior(real x, real beta) {
      return -log(beta); // log(1/beta) = log(1) - log(beta) = - log(beta)
  }
  
  
}
data { 
  int N;
  real y[N]; // points sampled from gpd in python with some(known) parameters, by mcmc we recover true values of those params
}
parameters { 
  real alpha;
  real beta;
}
model {
  // Priors; no priors - we assume improper priors on params
  alpha ~ gamma(1,1);
  beta ~ gamma(1,1);

// Likelihood
  for(n in 1:N) {
    target += myGPD_lpdf( y[n] | alpha, beta );
  }

}
generated quantities{}
"""

In [4]:
Fisher = """
functions { 
 real myFisher_lpdf(real y, real alpha1, real alpha2, real beta) {
      return -lbeta(alpha1,alpha2)-log(beta)+(alpha1-1)*log(y/beta)-(alpha1+alpha2)*log(1+y/beta);
  }
  
// to recover more general distribution of Fisher parametrized by three parameters we need to multiply the above distribution 
// by: df1**df1/2
// we have alpha1,2 = df1,2/2, beta = df2/df1
}

data { 
  int N;
  real y[N]; // points sampled from fisher in python with some(known) parameters, by mcmc we recover true values of those params
}
parameters { 
  //parameters of the Fisher
  //real df1;
  //real df2;
  real<lower=0> alpha1;
  real<lower=0> alpha2;
  real<lower=0> beta;
  
}
model {
  // when we deliberately do not specify priors then Stan works with improper priors
  alpha1 ~ gamma(1,1);
  alpha2 ~ gamma(1,1);
  beta ~ 1/beta;  //gamma(1,1);
   // Likelihood
  for(n in 1:N) {
    target += myFisher_lpdf( y[n] |alpha1, alpha2, beta);
  }
}

generated quantities{}
"""

In [ ]:
data = dict(N = k,  y = frechet) 
fit = StanModel(model_code=GPD).sampling(data=data,iter=1000,warmup=200, chains=1) #we sample from the provided data ;
print(fit)

In [ ]:
traceplot_beta_GPD = list(fit.extract().values())[1].tolist() 
traceplot_alpha = list(fit.extract().values())[0].tolist()
traceplot_gamma = np.divide(np.ones(len(traceplot_alpha)), traceplot_alpha)
beta_GPD = np.mean(list(fit.extract().values())[1].tolist())
alpha = np.mean(list(fit.extract().values())[0].tolist())
gamma = 1 / alpha 
print(" alpha = ", alpha, "\n beta = ", beta, "\n gamma = ", gamma)


In [ ]:
distr = "GPD(alpha, beta)"
plot(traceplot_alpha, "alpha", distr)
plot(traceplot_beta_GPD, "beta", distr)

In [ ]:
# check different levels of quantiles to compare them with the original ones in the pretty table above

In [ ]:
data = dict(N = k,  y = frechet) 
fit = StanModel(model_code=Fisher).sampling(data=data,iter=1000,warmup=200, chains=1) #we sample from the provided data ;
print(fit)

In [ ]:
traceplot_beta = list(fit.extract().values())[2].tolist()
traceplot_alpha1 = list(fit.extract().values())[1].tolist()
traceplot_alpha2 = list(fit.extract().values())[0].tolist()
beta = np.mean(list(fit.extract().values())[2].tolist())
alpha2 = np.mean(list(fit.extract().values())[1].tolist())
alpha1 = np.mean(list(fit.extract().values())[0].tolist())
print(" alpha1 = ", alpha1, "\n alpha2 = ", alpha2, "\n beta = ", beta)
# gamma = 1 / alpha 

In [ ]:
distr = "Fischer(alpha1, alpha2, beta)"
plot(traceplot_alpha1, "alpha1", distr)
plot(traceplot_alpha2, "alpha2", distr)
plot(traceplot_beta, "beta", distr)

In [ ]:
# once again we plot quantiles from the beginning to be able to compare
print(t)
# compare with the theeratical

In [ ]:
# theoretical quantiles 

In [ ]:
# we check the values we will use to obtain quantiles 
# values_GPD = [" N = ", N," k = ",  k," beta = ",  beta," gamma = ",  gamma, " u = ", u]

# q = [0.925, 0.95, 0.975, 0.99, 0.999]
# quant_GPD = np.zeros(len(q)) 
# for i in range(len(q)):
#     quant_GPD[i] = u + beta_GPD*( pow( N * (1-q[i]) / k, -gamma ) - 1 ) 
# print(quant_GPD)


In [ ]:
# p = [0.025, 0.25, 0.5, 0.75, 0.975]# to check other values such as p = 0.025, 0.25, 0.5, 0.75, 0.975
quant_Fischer = np.zeros(len(q))
beta0 = alpha2 / alpha1

for i in range(len(q)):
#     q[i] = N * p[i] / k
#     print(q[i])
    quant_Fischer[i] = u + beta0 / beta * f.isf(N / k * (1-q[i]), 2 * alpha1, 2 * alpha2, loc=0, scale=1)
print(quant_Fischer)

In [ ]:
# print(" beta0 = ", beta0, "\n beta = ", beta, "\n beta0 / beta = " , beta0/beta, "\n beta / beta0 = ", beta/beta0)

In [ ]:
# now if the quantiles are correct we need to average the results

In [ ]:
# --------------------------------------------------------------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------
# Bayesian way of obtaining the quantiles, we start with quantile estimation for GPD

In [5]:
# for each data set r_i, i \in 1, ..., 20 fit GPD and using parameters obtained calculate the quantiles of bayes_GPD, GPD, 
# for each data set r_i, i \in 1, ..., 20 fit Fisher and using parameters obtained calculate the quantiles of bayes_Fisher, Fisher, 


def quantiles_GPD(r): 
    """ 
    r is the data sample
    """
    q = [0.9, 0.95, 0.975, 0.99, 0.999]
    quant_GPD = np.zeros(len(q)) 
    bayesian_quant_GPD = np.zeros(len(q))
    
    k = 100 # number of excesses
    frechet, u = k_greatest_values(r, k)
    
    # here we fit GPD to excesses via PyStan
    data = dict(N = k,  y = frechet) 
    fit = StanModel(model_code=GPD).sampling(data=data,iter=1000,warmup=200, chains=1) 
    
    # we save the params from the fit to calculate GPD quantiles and their traceplots to calculate Bayesian GPD quantiles
    traceplot_beta_GPD = list(fit.extract().values())[1].tolist() 
    traceplot_alpha = list(fit.extract().values())[0].tolist()
    traceplot_gamma = np.divide(np.ones(len(traceplot_alpha)), traceplot_alpha)
    beta_GPD = np.mean(list(fit.extract().values())[1].tolist())
    alpha = np.mean(list(fit.extract().values())[0].tolist())
    gamma = 1 / alpha 
    
    # we also want to keep track of parameters from each fit
#     values_of_beta_GPD = np.zeros(n)
    
    for i in range(len(q)):
        quant_GPD[i] = u + beta_GPD*( pow( N * (1-q[i]) / k, -gamma ) - 1 ) 
        for j in range(len(traceplot_gamma)):
                bayesian_quant_GPD[i] = bayesian_quant_GPD[i] + u + traceplot_beta_GPD[j] * (pow( N * (1 - q[i]) / k, - traceplot_gamma[j] ) - 1)
    bayesian_quant_GPD = bayesian_quant_GPD / len(traceplot_gamma)
    list_of_params = [alpha, beta_GPD]
    return(quant_GPD, bayesian_quant_GPD, list_of_params ) # it return arrays: quant_GPD, bayesian_quant_GPD and values alpha, beta_GPD

# now the same as above but for Fisher quantiles
def quantiles_Fisher(r):
    q = [0.9, 0.95, 0.975, 0.99, 0.999]
    quant_Fisher = np.zeros(len(q)) 
    bayesian_quant_Fisher = np.zeros(len(q))
    
    k = 100 # number of excesses
    frechet, u = k_greatest_values(r, k)
    
    # here we fit GPD to excesses via PyStan
    data = dict(N = k,  y = frechet) 
    fit = StanModel(model_code=Fisher).sampling(data=data,iter=1000,warmup=200, chains=1) 
    
    # we save the params from the fit to calculate Fisher quantiles and their traceplots to calculate Bayesian Fisher quantiles
    traceplot_beta = list(fit.extract().values())[2].tolist()
    traceplot_alpha1 = list(fit.extract().values())[1].tolist()
    traceplot_alpha2 = list(fit.extract().values())[0].tolist()
    beta = np.mean(list(fit.extract().values())[2].tolist())
    alpha2 = np.mean(list(fit.extract().values())[1].tolist())
    alpha1 = np.mean(list(fit.extract().values())[0].tolist())
    beta0 = alpha2/alpha1
    
    for i in range(len(q)):
        quant_Fisher[i] = u + beta0 / beta * f.isf(N / k * (1-q[i]), 2 * alpha1, 2 * alpha2, loc=0, scale=1)
        for j in range(len(traceplot_alpha1)):
                bayesian_quant_Fisher[i] = bayesian_quant_Fisher[i] + u + traceplot_alpha2[j] / traceplot_alpha1[j] / traceplot_beta[j] * f.isf(N / k *(1- q[i]), 2 * traceplot_alpha1[j], 2 * traceplot_alpha2[j], loc=0, scale=1)
    bayesian_quant_Fisher = bayesian_quant_Fisher / len(traceplot_alpha1)
    return(quant_Fisher, bayesian_quant_Fisher, alpha1, alpha2, beta) # it return arrays: quant_Fisher, bayesian_quant_Fisher and values of params
    


In [6]:
# check what does the function returns, maybe there is no need to call it several times and we can get by with just one call?
N, beta = 1000,  1/2
q = [0.9, 0.95, 0.975, 0.99, 0.999]
n = 1 # number of sampled dataset over which we average the quantiles
averaged_quant_GPD, averaged_bayesian_quant_GPD, params_GPD = np.ones(len(q)), np.ones(len(q)), np.ones(2) # initialize the values of quantiles
# averaged_quant_Fisher, averaged_bayesian_quant_Fisher = np.zeros(len(q)), np.zeros(len(q))
# values_alpha_GPD, values_beta_GPD, values_alpha1_Fisher, values_alpha2_Fisher, values_beta_Fisher = np.ones(n), np.ones(n), np.zeros(n), np.zeros(n), np.zeros(n)


In [7]:
print(averaged_quant_GPD, averaged_bayesian_quant_GPD, params_GPD )

[1. 1. 1. 1. 1.] [1. 1. 1. 1. 1.] [1. 1.]


In [8]:
r = frechet.rvs(beta, size=N)
k = 100 # number of excesses
frechet, u = k_greatest_values(r, k)

new_averaged_quant_GPD, new_averaged_bayesian_quant_GPD, params_GPD = quantiles_GPD(r)
# averaged_quant_GPD, averaged_bayesian_quant_GPD, values_alpha_GPD, values_beta_GPD += new_averaged_quant_GPD, new_averaged_bayesian_quant_GPD, new_values_alpha_GPD, new_values_beta_GPD

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_1561b2fac3108872ed6bafb6d2492041 NOW.
C:\Users\Michal Lewandowski\Anaconda3\envs\py35\lib\site-packages\pystan\misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


In [11]:
print(averaged_quant_GPD, averaged_bayesian_quant_GPD, params_GPD )
params_GPD[1]

[1. 1. 1. 1. 1.] [1. 1. 1. 1. 1.] {1.9942238730065611, 2.9728366993225666}


TypeError: 'set' object does not support indexing

In [ ]:
# ok, so there is no need to call the function several times using quantiles_GPD(r)[1]
# now: can we add those values to the ones already existing, not just obtain them by initialization ?

In [ ]:
# for bigger sample to make averaging over numeroues datasets I use the loop for

In [ ]:
N, beta = 1000,  1/2
q = [0.9, 0.95, 0.975, 0.99, 0.999]
n = 2 # number of sampled dataset over which we average the quantiles
averaged_quant_GPD, averaged_bayesian_quant_GPD = np.zeros(len(q)), np.zeros(len(q)) # initialize the values of quantiles
averaged_quant_Fisher, averaged_bayesian_quant_Fisher = np.zeros(len(q)), np.zeros(len(q))
values_alpha_GPD, values_beta_GPD, values_alpha1_Fisher, values_alpha2_Fisher, values_beta_Fisher = np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n)
for i in range(n):
    r = frechet.rvs(beta, size=N)
    # save the values of results of quantiles funciotns instead of calling them two times !
    
    averaged_quant_GPD  = averaged_quant_GPD + quantiles_GPD(r)[0]
    averaged_bayesian_quant_GPD =  averaged_bayesian_quant_GPD + quantiles_GPD(r)[1]
    values_alpha_GPD[i], values_beta_GPD[i] = quantiles_GPD(r)[2], quantiles_GPD(r)[3]
    
    averaged_quant_Fisher = averaged_quant_Fisher + quantiles_Fisher(r)[0] 
    averaged_bayesian_quant_Fisher = averaged_bayesian_quant_Fisher + quantiles_Fisher(r)[1]
    values_alpha1_Fisher[i], values_alpha2_Fisher[i], values_beta_Fisher[i] = quantiles_Fisher(r)[2], quantiles_Fisher(r)[3], quantiles_Fisher(r)[4]

averaged_quant_GPD, averaged_bayesian_quant_GPD = averaged_quant_GPD / n, averaged_bayesian_quant_GPD / n
averaged_quant_Fisher, averaged_bayesian_quant_Fisher = averaged_quant_Fisher / n, averaged_bayesian_quant_Fisher / n

In [ ]:
quant_th = np.ones(len(q))
for i in range(len(quant_th)):
    quant_th[i] = pow(-log(q[i]), -1/2)
print(quant_th)

In [ ]:
print(averaged_quant_GPD,"\n",  averaged_bayesian_quant_GPD, "\n", averaged_quant_Fisher, "\n", averaged_bayesian_quant_Fisher)

In [ ]:
# save parameters of GPD and Fisher for every fit 

# bayesian_quantile_gpd = np.zeros(len(q))

# for j in range(len(q)):
#     for i in range(len(traceplot_gamma)):
#         bayesian_quantile_gpd[j] = bayesian_quantile_gpd[j] + u + traceplot_beta_GPD[i] * (pow( N * (1 - q[j]) / k, - traceplot_gamma[i] ) - 1)
# bayesian_quantile_gpd = bayesian_quantile_gpd / len(traceplot_gamma)
# bayesian_quantile_gpd

In [ ]:
# bayesian_quantile_fisher = np.zeros(len(q))
# # u + beta0 / beta * f.isf(N / k * (1-q[i]), 2 * alpha1, 2 * alpha2, loc=0, scale=1)
# for j in range(len(q)):
#     for i in range(len(traceplot_alpha1)):
#         bayesian_quantile_fisher[j] = bayesian_quantile_fisher[j] + u + traceplot_alpha2[i] / traceplot_alpha1[i] / traceplot_beta[i] * f.isf(N / k *(1- q[j]), 2 * traceplot_alpha1[i], 2 * traceplot_alpha2[i], loc=0, scale=1)
# bayesian_quantile_fisher = bayesian_quantile_fisher / len(traceplot_alpha1)
# bayesian_quantile_fisher

In [ ]:
# '0.9', '0.95', '0.975', '0.99', '0.999'

t_quantiles = PrettyTable(['how obtained',  'mean', 'sd', '92.5%', '95%', '97.5%', '99%', '99.9%'])
# t_quantiles.add_row(['numpy', 
#            "%.3f" % round(np.mean(r),m),
#            "%.3f" % round(np.std(r),m),
#            "%.3f" % round(np.percentile(r, 92.5),m), 
#            "%.3f" % round(np.percentile(r, 95),m), 
#            "%.3f" % round(np.percentile(r, 97.5),m), 
#            "%.3f" % round(np.percentile(r, 99),m),
#            "%.3f" % round(np.percentile(r, 99.9),m)])
t_quantiles.add_row(['theoretically', '--', '--',
                     "%.3f" % round(quant_th[0],m),  
                     "%.3f" % round(quant_th[1], m), 
                     "%.3f" % round(quant_th[2], m),
                     "%.3f" % round(quant_th[3], m),
                     "%.3f" % round(quant_th[4], m) ])
t_quantiles.add_row(['Bayes Fisher',  '--', '--',
                     "%.3f" % round(bayesian_quantile_fisher[0],m),  
                     "%.3f" % round(bayesian_quantile_fisher[1], m), 
                     "%.3f" % round(bayesian_quantile_fisher[2], m),
                     "%.3f" % round(bayesian_quantile_fisher[3], m),
                     "%.3f" % round(bayesian_quantile_fisher[4], m) ])
t_quantiles.add_row([' Fisher',  '--', '--',
                     "%.3f" % round(quant_Fischer[0],m),  
                     "%.3f" % round(quant_Fischer[1], m), 
                     "%.3f" % round(quant_Fischer[2], m),
                     "%.3f" % round(quant_Fischer[3], m),
                     "%.3f" % round(quant_Fischer[4], m) ])
t_quantiles.add_row([' Bayes GPD', '--', '--',
                     "%.3f" % round(bayesian_quantile_gpd[0],m),  
                     "%.3f" % round(bayesian_quantile_gpd[1], m), 
                     "%.3f" % round(bayesian_quantile_gpd[2], m),
                     "%.3f" % round(bayesian_quantile_gpd[3], m),
                     "%.3f" % round(bayesian_quantile_gpd[4], m) ])
t_quantiles.add_row(['  GPD',  '--', '--',
                     "%.3f" % round(quant_GPD[0],m),  
                     "%.3f" % round(quant_GPD[1], m), 
                     "%.3f" % round(quant_GPD[2], m),
                     "%.3f" % round(quant_GPD[3], m),
                     "%.3f" % round(quant_GPD[4], m) ])
# t.add_row(['Bob', 19])
print(t_quantiles)